In [ ]:
from abtem import GridScan, PixelatedDetector, Potential, Probe, show_atoms, SMatrix, AnnularDetector, FrozenPhonons
from abtem.detect import PixelatedDetector
from abtem.reconstruct import MixedStatePtychographicOperator, RegularizedPtychographicOperator
from abtem.measure import Measurement, Calibration, bandlimit, center_of_mass
from abtem.utils import energy2wavelength
from abtem.transfer import CTF, scherzer_defocus
from abtem.structures import orthogonalize_cell
from abtem.noise import poisson_noise
from ase.build import mx2
from ase.io import read
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Times New Roman'
import tifffile

# Atomic structure

In [ ]:
atoms = mx2(formula='MoS2', kind='2H', a=3.18, thickness=3.19, size=(1, 1, 1), vacuum=None)
atoms = orthogonalize_cell(atoms)
atoms.center(vacuum=2, axis=2)

atoms *= (5,3,1)
#del atoms[16]


fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12,4))

show_atoms(atoms, ax=ax1, title='Top view', numbering=False)
show_atoms(atoms, ax=ax2, plane='xz', title='Side view')
plt.show()

# Generation of the projected atomic potential
![alt text](image/atomic_potential.jpg "practice")

In [ ]:
potential = Potential(atoms, 
                      sampling=.05,
                      projection='infinite', 
                      slice_thickness=1, 
                      parametrization='kirkland').build()

fig, ax1 = plt.subplots(1, 1, figsize=(5, 5))
potential.project().show(ax=ax1, cmap="inferno")
plt.show()

![alt text](image/atomic_potential_2.jpg "practice")

# Reflection of thermal diffuse scattering (frozen phonon)

In [ ]:
frozen_phonons = FrozenPhonons(atoms, 16, {"Mo":0.05, "S":0.05}, seed=56)
atoms_conf = next(iter(frozen_phonons))

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

show_atoms(atoms_conf, ax=ax1, title='Top view')
show_atoms(atoms_conf, ax=ax2, plane='xz', title='Side view - xz')
plt.show()

In [ ]:
tds_potential = Potential(frozen_phonons, 
                      sampling=.05,
                      projection='infinite', 
                      slice_thickness=1, 
                      parametrization='kirkland').build()

print(tds_potential.gpts, tds_potential.extent, tds_potential.sampling)

tfig, ax1 = plt.subplots(1, 1, figsize=(5, 5))
tds_potential.project().show(ax=ax1, cmap="inferno")
plt.show()

# Probe generation
![alt text](image/STEM_probe.jpg "practice")
![alt text](image/STEM_probe_2.jpg "practice")

In [ ]:
energy = 80E3
C3 = 1E6 # 1E6 angstrom = 0.1 mm
aperture_semiangle = 30 # mrad
sch_defocus = scherzer_defocus(C3, energy)
print(sch_defocus)

In [ ]:
#ctf   = CTF(parameters={'C10': sch_defocus*0.9, 'C30':C3}, semiangle_cutoff=aperture_semiangle)
#probe = Probe(semiangle_cutoff=aperture_semiangle, energy=energy, ctf=ctf)

probe = Probe(semiangle_cutoff=aperture_semiangle, energy=energy, Cs=C3, defocus=sch_defocus*1.1)
probe.grid.match(tds_potential)

print(tds_potential.gpts, tds_potential.extent, tds_potential.sampling)
print(probe.gpts, probe.extent, probe.sampling)

print("wavelength", probe.wavelength)
print("Nyquist sampling", probe.ctf.nyquist_sampling)

fig, axes = plt.subplots(1, 2, figsize=(10, 5))
probe.show(ax=axes[0])
probe.show(ax=axes[1], power=0.5)
plt.show()

![alt text](image/Nyquist_Shanon_sampling_theorem.gif "practice")

In [ ]:
gridscan = GridScan((0,0), np.array(tds_potential.extent), sampling=0.5)
print(gridscan.gpts)

# 4D-STEM simulation
![alt text](image/exit_wave.jpg "practice")

In [ ]:
detector = PixelatedDetector(max_angle = aperture_semiangle*3.0, resample=(2.0, 2.0))
measurement = probe.scan(gridscan, [detector], tds_potential)

print(measurement.shape)
for i in range(measurement.dimensions):
    print(measurement.calibrations[i].name, measurement.calibrations[i].units, measurement.calibrations[i].sampling)

In [ ]:
bright_detector = AnnularDetector(inner=10, outer=30)
bright_measurement = bright_detector.integrate(measurement)

maadf_detector = AnnularDetector(inner=50, outer=90)
maadf_measurement = maadf_detector.integrate(measurement)

pacbed = measurement.mean(axis=(0, 1))

fig, axes = plt.subplots(1, 4, figsize=(20, 5))
measurement.show(ax=axes[0], cmap="inferno", power=0.5)
pacbed.show(ax=axes[1], cmap="inferno", power=0.1)
bright_measurement.show(ax=axes[2])
maadf_measurement.show(ax=axes[3])
fig.tight_layout()
plt.show()

# Differential phase contrast imaging
![alt text](image/DPC.jpg "practice")

In [ ]:
com_x, com_y = center_of_mass(measurement)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

com_x.interpolate(.1).show(ax=ax1)
com_y.interpolate(.1).show(ax=ax2)
fig.tight_layout()
plt.show()

In [ ]:
icom = center_of_mass(measurement, return_icom=True)

icom.show()
plt.show()

# (regularized) Ptychographical iterative engine (PIE)
![alt text](image/rpie.jpg "practice")

## Bright-field (BF) only

In [ ]:
bf_measurement = bandlimit(measurement, aperture_semiangle)
bf_measurement.show(cmap="inferno", power=0.5)
plt.show()

In [ ]:
n_iter = 10
experimental_ptycho_operator = RegularizedPtychographicOperator(bf_measurement,
                                                               semiangle_cutoff=aperture_semiangle,
                                                               energy=energy,
                                                               parameters={'object_px_padding':(16, 16)}).preprocess()

exp_objects, exp_probes, exp_positions, exp_sse  = experimental_ptycho_operator.reconstruct(
    max_iterations = n_iter,
    random_seed=1,
    return_iterations=True,
    parameters={'alpha':1.0,
                'beta':1.0})

In [ ]:
print(exp_objects[-1].shape)
for i in range(exp_objects[-1].dimensions):
    print(exp_objects[-1].calibrations[i].name, exp_objects[-1].calibrations[i].units,
          exp_objects[-1].calibrations[i].sampling)

In [ ]:
plot_every = int(n_iter/5)

fig, axes = plt.subplots(2, int(np.ceil(len(exp_objects) / plot_every))+1, figsize=(20, 8))

for i, j in enumerate(range(0,len(exp_objects), plot_every)):
    axes[0,i].imshow(np.angle(exp_objects[j].array).T, origin='lower', cmap='gray')
    axes[0,i].set_title('iteration: %d, SSE: %.2e'%(j+1, exp_sse[j]))
    axes[1,i].imshow(np.abs(exp_probes[j].array).T, origin='lower', cmap='gray')
    axes[0,i].axis("off")
    axes[1,i].axis("off")

axes[0,-1].imshow(np.angle(exp_objects[-1].array).T, origin='lower', cmap='gray')
axes[0,-1].set_title('iteration: %d, SSE: %.2e'%(n_iter, exp_sse[-1]))
axes[1,-1].imshow(np.abs(exp_probes[-1].array).T, origin='lower', cmap='gray')
axes[0,-1].axis("off")
axes[1,-1].axis("off")

fig.tight_layout()
plt.show()

In [ ]:
fig, axd = plt.subplots(1, 2, figsize=(20, 10))
exp_objects[-1].angle().show(ax=axd[0], title=f"SSE = {float(exp_sse[-1]):.3e}", cmap='inferno')
exp_probes[-1].intensity().show(ax=axd[1], cmap="gray", power=0.3)
fig.tight_layout()
plt.show()

![alt text](image/pie_condition.jpg "practice")

## BF + dark-field scattering

In [ ]:
band_limited_measurment = bandlimit(measurement, aperture_semiangle*2.0)
band_limited_measurment.show(cmap="inferno", power=0.5)
plt.show()

In [ ]:
tifffile.imwrite("4DSTEM_measurement.tif", band_limited_measurement.array)

In [ ]:
n_iter = 10
experimental_ptycho_operator = RegularizedPtychographicOperator(band_limited_measurment,
                                                               semiangle_cutoff=aperture_semiangle,
                                                               energy=energy,
                                                               parameters={'object_px_padding':(10, 10)}).preprocess()

exp_objects, exp_probes, exp_positions, exp_sse  = experimental_ptycho_operator.reconstruct(
    max_iterations = n_iter,
    random_seed=1,
    return_iterations=True,
    parameters={'alpha':1.0,
                'beta':1.0})

In [ ]:
print(exp_objects[-1].shape)
for i in range(exp_objects[-1].dimensions):
    print(exp_objects[-1].calibrations[i].name, exp_objects[-1].calibrations[i].units,
          exp_objects[-1].calibrations[i].sampling)

In [ ]:
plot_every = int(n_iter/5)

fig, axes = plt.subplots(2, int(np.ceil(len(exp_objects) / plot_every))+1, figsize=(20, 8))

for i, j in enumerate(range(0,len(exp_objects), plot_every)):
    axes[0,i].imshow(np.angle(exp_objects[j].array).T, origin='lower', cmap='gray')
    axes[0,i].set_title('iteration: %d, SSE: %.2e'%(j+1, exp_sse[j]))
    axes[1,i].imshow(np.abs(exp_probes[j].array).T, origin='lower', cmap='gray')
    axes[0,i].axis("off")
    axes[1,i].axis("off")

axes[0,-1].imshow(np.angle(exp_objects[-1].array).T, origin='lower', cmap='gray')
axes[0,-1].set_title('iteration: %d, SSE: %.2e'%(n_iter, exp_sse[-1]))
axes[1,-1].imshow(np.abs(exp_probes[-1].array).T, origin='lower', cmap='gray')
axes[0,-1].axis("off")
axes[1,-1].axis("off")

fig.tight_layout()
plt.show()

In [ ]:
fig, axd = plt.subplots(1, 2, figsize=(20, 10))
exp_objects[-1].angle().show(ax=axd[0], title=f"SSE = {float(exp_sse[-1]):.3e}", cmap='inferno')
exp_probes[-1].intensity().show(ax=axd[1], cmap="gray", power=0.5)
fig.tight_layout()
plt.show()

## Low-dose + noise

In [ ]:
pn_measurement = poisson_noise(measurement, 1E5) # electrons per A^2
pn_bright_measurement = bright_detector.integrate(pn_measurement)
pn_maadf_measurement = maadf_detector.integrate(pn_measurement)

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(30, 10))
pn_measurement.show(ax=axes[0], cmap="inferno", power=0.5)
pn_bright_measurement.show(ax=axes[1])
pn_maadf_measurement.show(ax=axes[2])
plt.show()

In [ ]:
band_limited_measurment = bandlimit(pn_measurement, aperture_semiangle*2.0)
band_limited_measurment.show(cmap="inferno", power=0.5)
plt.show()

In [ ]:
n_iter = 10
experimental_ptycho_operator = RegularizedPtychographicOperator(band_limited_measurment,
                                                               semiangle_cutoff=aperture_semiangle,
                                                               energy=energy,
                                                               parameters={'object_px_padding':(10, 10)}).preprocess()

exp_objects, exp_probes, exp_positions, exp_sse  = experimental_ptycho_operator.reconstruct(
    max_iterations = n_iter,
    random_seed=1,
    return_iterations=True,
    parameters={'alpha':1.0,
                'beta':1.0})

In [ ]:
print(exp_objects[-1].shape)
for i in range(exp_objects[-1].dimensions):
    print(exp_objects[-1].calibrations[i].name, exp_objects[-1].calibrations[i].units,
          exp_objects[-1].calibrations[i].sampling)

In [ ]:
plot_every = int(n_iter/5)

fig, axes = plt.subplots(2, int(np.ceil(len(exp_objects) / plot_every))+1, figsize=(20, 8))

for i, j in enumerate(range(0,len(exp_objects), plot_every)):
    axes[0,i].imshow(np.angle(exp_objects[j].array).T, origin='lower', cmap='gray')
    axes[0,i].set_title('iteration: %d, SSE: %.2e'%(j+1, exp_sse[j]))
    axes[1,i].imshow(np.abs(exp_probes[j].array).T, origin='lower', cmap='gray')
    axes[0,i].axis("off")
    axes[1,i].axis("off")

axes[0,-1].imshow(np.angle(exp_objects[-1].array).T, origin='lower', cmap='gray')
axes[0,-1].set_title('iteration: %d, SSE: %.2e'%(n_iter, exp_sse[-1]))
axes[1,-1].imshow(np.abs(exp_probes[-1].array).T, origin='lower', cmap='gray')
axes[0,-1].axis("off")
axes[1,-1].axis("off")

fig.tight_layout()
plt.show()

In [ ]:
fig, axd = plt.subplots(1, 2, figsize=(20, 10))
exp_objects[-1].angle().show(ax=axd[0], title=f"SSE = {float(exp_sse[-1]):.3e}", cmap='inferno')
exp_probes[-1].intensity().show(ax=axd[1], cmap="gray", power=0.5)
fig.tight_layout()
plt.show()